<hr>
<hr>

# Libraries

In [119]:
import pandas as pd
import ast
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

In [120]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

<hr>
<hr>

# Dataset

In [121]:
df = pd.read_csv("secondAttempt_final_df_merged.csv")

* df['y1'] = Kellgren-Lawrence (KL) grades [0,1,2,3,4] (V00XRKL)

* df['y2'] = Knee osteoarthritis (OA), A=non-OA and B=OA (V03KL)

In [122]:
y1_labels = df['y1'].unique().tolist()
y1_labels.sort()
print(y1_labels)
print(type(y1_labels[0]))

[0, 1, 2, 3, 4]
<class 'int'>


In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   case      196 non-null    int64  
 1   fv1_mean  196 non-null    float64
 2   fv2_mean  196 non-null    float64
 3   fv3_mean  196 non-null    float64
 4   y1        196 non-null    int64  
 5   y2        196 non-null    object 
dtypes: float64(3), int64(2), object(1)
memory usage: 9.3+ KB


In [125]:
df.head()

,case,fv1_mean,fv2_mean,fv3_mean,y1,y2
0,9002116,75.003809,108.119251,96.902299,3,B
1,9005075,98.129495,113.714378,106.785496,0,A
2,9005132,110.778392,124.761203,112.053324,4,B
3,9026934,96.534927,99.721493,86.627444,2,A
4,9030718,90.302990,104.339804,119.286584,3,B
